In [344]:
import json
from math import tanh
import random
def data_équipes_dict():
    équipes = {}
    with open("data_équipes.csv", "r", encoding="utf8") as f_data_equipes:
        data_equipes = [line.strip().split(";") for line in f_data_equipes][1:]
    for e in data_equipes:
        équipe = {}
        équipe["gardiens"] = json.loads(e[1])
        équipe["défenseurs"] = json.loads(e[2])
        équipe["milieux"] = json.loads(e[3])
        équipe["attaquants"] = json.loads(e[4])
        équipes[e[0]] = équipe
    
    return équipes


def data_joueurs_dict(data_csv):
    with open(data_csv, "r", encoding="utf8") as f_data_joueurs:
        data_joueurs = [line.strip().split(",") for line in f_data_joueurs][1:]

    joueurs = {}
    for j in data_joueurs:
        joueur = {}
        joueur["nationalité"] = j[1]
        joueur["rating"] = j[2]
        joueur["âge"] = j[3]
        joueurs[j[0]] = joueur
        
    return joueurs


def get_nationalité(L, afficher=True):
    nationnalité = {}
    for nom in équipes:
        for poste in ["gardiens","défenseurs", "milieux", "attaquants"]:
            for j in équipes[nom][poste]:
                if j in L:
                    if afficher:
                        print(j, nom)
                    nationnalité[nom] = nationnalité.get(nom, 0) + 1

    return nationnalité

In [2]:
équipes = data_équipes_dict()

In [3]:
joueurs = data_joueurs_dict("data_joueurs_new.csv")

In [14]:
équipes["France"]["gardiens"]

['Hugo Lloris', 'Steve Mandanda', 'Alphonse Areola']

In [5]:
joueurs

{'Kylian Mbappé': {'nationalité': 'France', 'rating': '91', 'âge': '23'},
 'Kevin De Bruyne': {'nationalité': 'Belgique', 'rating': '91', 'âge': '31'},
 'Robert Lewandowski': {'nationalité': 'Pologne', 'rating': '91', 'âge': '34'},
 'Virgil van Dijk': {'nationalité': 'Pays-Bas', 'rating': '90', 'âge': '31'},
 'Thibaut Courtois': {'nationalité': 'Belgique', 'rating': '90', 'âge': '30'},
 'Cristiano Ronaldo': {'nationalité': 'Portugal', 'rating': '90', 'âge': '37'},
 'Alisson': {'nationalité': 'Brésil', 'rating': '89', 'âge': '30'},
 'Joshua Kimmich': {'nationalité': 'Allemagne', 'rating': '89', 'âge': '27'},
 'Ederson': {'nationalité': 'Brésil', 'rating': '89', 'âge': '29'},
 'Sadio Mané': {'nationalité': 'Sénégal', 'rating': '89', 'âge': '30'},
 'Harry Kane': {'nationalité': 'Angleterre', 'rating': '89', 'âge': '29'},
 'Jan Oblak': {'nationalité': 'Slovénie', 'rating': '89', 'âge': '29'},
 'Casemiro': {'nationalité': 'Brésil', 'rating': '89', 'âge': '30'},
 'Neymar Jr': {'nationalité':

In [95]:
class Joueur:
    def __init__(self, name, nationalité, rating, age):
        self.name = name
        self.nationalité = nationalité
        self.rating = int(rating)
        self.age = int(age)

In [96]:
j = Joueur("Kylian Mbappé", "France", 91, 23)

In [251]:
class Equipe:
    def __init__(self, nom, gardiens, défenseurs, milieux, attaquants):
        self.nom = nom
        self.gardiens = gardiens
        self.défenseurs = défenseurs
        self.milieux = milieux
        self.attaquants = attaquants
        self.joueurs = gardiens + défenseurs + milieux + attaquants
        self.effectif = len(self.joueurs)
        self.rang = sum([j.rating for j in self.joueurs]) / self.effectif
        
    def max_poste(self, L):
        ratings = [j.rating for j in L]
        i = ratings.index(max(ratings))
        return L[i]
        
    def selection(self, formation="442"):
        joueurs_liste = []
        nb_d, nb_m, nb_a = [int(e) for e in list(formation)]
        
        joueurs_liste.append(self.max_poste(self.gardiens))
        
        LL = self.défenseurs.copy()
        for _ in range(nb_d):
            j = self.max_poste(LL)
            joueurs_liste.append(j)
            LL.remove(j)
            
        LL = self.milieux.copy()
        for _ in range(nb_m):
            j = self.max_poste(LL)
            joueurs_liste.append(j)
            LL.remove(j)
            
        LL = self.attaquants.copy()
        for _ in range(nb_a):
            j = self.max_poste(LL)
            joueurs_liste.append(j)
            LL.remove(j)
            
            
        
        return joueurs_liste

In [706]:
class Equipes():
    def __init__(self, équipes):
        self.équipes = [créer_équipe(e) for e in équipes]
    
    def créer_équipe(nom):
        l_g = [Joueur(j, joueurs[j]["nationalité"], joueurs[j]["rating"], joueurs[j]["âge"]) for j in équipes[nom]["gardiens"]]
        l_d = [Joueur(j, joueurs[j]["nationalité"], joueurs[j]["rating"], joueurs[j]["âge"]) for j in équipes[nom]["défenseurs"]]
        l_m = [Joueur(j, joueurs[j]["nationalité"], joueurs[j]["rating"], joueurs[j]["âge"]) for j in équipes[nom]["milieux"]]
        l_a = [Joueur(j, joueurs[j]["nationalité"], joueurs[j]["rating"], joueurs[j]["âge"]) for j in équipes[nom]["attaquants"]]
        return Equipe(nom, l_g, l_d, l_m, l_a)
    
    def afficher_équipes(self):
        return [e.nom for e in self.équipes]

In [741]:
class Poule:
    def __init__(self, nom):
        self.nom = nom
        self.équipes_poule = []
        self.nombres_équipes = 4
        
    def ajouter_équipe_poule(self, nom_équipe):
        if len(self.équipes_poule) < self.nombres_équipes:
            self.équipes_poule.append(nom_équipe)
            return True
        else:
            return False

class Poules:
    def __init__(self, équipes):
        self.équipes_mélangés = équipes.équipes.copy()
        random.shuffle(self.équipes_mélangés)
        self.nom_poules = ["A", "B", "C", "D", "E", "F", "G", "H"]
        self.poules = {}
    
    def création_poules(self):
        for n in self.nom_poules:
            p = Poule(n)
            for e in self.équipes_mélangés:
                if p.ajouter_équipe_poule(e):
                    print("Ajout équipe", e.nom)
                    self.équipes_mélangés.remove(e)
            self.poules[n] = p
                    

In [742]:
équipes_objet.afficher_équipes()

['Qatar',
 'Equateur',
 'Sénégal',
 'Pays-Bas',
 'Angleterre',
 'Iran',
 'Etats-Unis',
 'Pays de Galles',
 'Argentine',
 'Arabie saoudite',
 'Mexique',
 'Pologne',
 'France',
 'Australie',
 'Danemark',
 'Tunisie',
 'Espagne',
 'Costa Rica',
 'Allemagne',
 'Japon',
 'Belgique',
 'Canada',
 'Maroc',
 'Croatie',
 'Brésil',
 'Serbie',
 'Suisse',
 'Cameroun',
 'Portugal',
 'Ghana',
 'Uruguay',
 'Corée du Sud']

In [766]:
l_équipes = [e for e in équipes]
print(len(l_équipes))

32


In [754]:
équipes_objet = Equipes(l_équipes)

In [755]:
p = Poules(équipes_objet)

In [756]:
p.création_poules()

Ajout équipe Suisse
Ajout équipe Maroc
Ajout équipe Cameroun
Ajout équipe Allemagne
Ajout équipe Iran
Ajout équipe Angleterre
Ajout équipe Ghana
Ajout équipe Uruguay
Ajout équipe Corée du Sud
Ajout équipe Brésil
Ajout équipe Belgique
Ajout équipe Danemark
Ajout équipe Equateur
Ajout équipe Arabie saoudite
Ajout équipe Tunisie
Ajout équipe Qatar
Ajout équipe Australie
Ajout équipe Pays-Bas
Ajout équipe Argentine
Ajout équipe Canada
Ajout équipe Costa Rica
Ajout équipe Mexique
Ajout équipe France
Ajout équipe Pologne
Ajout équipe Espagne
Ajout équipe Sénégal
Ajout équipe Pays de Galles
Ajout équipe Croatie
Ajout équipe Etats-Unis
Ajout équipe Serbie


In [757]:
p.poules

{'A': <__main__.Poule at 0x1d9e81e7970>,
 'B': <__main__.Poule at 0x1d9e81e7bb0>,
 'C': <__main__.Poule at 0x1d9e81e7d60>,
 'D': <__main__.Poule at 0x1d9e81e7220>,
 'E': <__main__.Poule at 0x1d9e81e7ac0>,
 'F': <__main__.Poule at 0x1d9e81e7d90>,
 'G': <__main__.Poule at 0x1d9e81e7e20>,
 'H': <__main__.Poule at 0x1d9e81e7ee0>}

In [765]:
for kp, vp in p.poules.items():
    print(kp)
    for e in vp.équipes_poule:
        print(e.nom)
    print()

A
Suisse
Maroc
Cameroun
Allemagne

B
Iran
Angleterre
Ghana
Uruguay

C
Corée du Sud
Brésil
Belgique
Danemark

D
Equateur
Arabie saoudite
Tunisie
Qatar

E
Australie
Pays-Bas
Argentine
Canada

F
Costa Rica
Mexique
France
Pologne

G
Espagne
Sénégal
Pays de Galles
Croatie

H
Etats-Unis
Serbie



In [265]:
équipes["France"]

{'gardiens': ['Hugo Lloris', 'Steve Mandanda', 'Alphonse Areola'],
 'défenseurs': ['Benjamin Pavard',
  'Axel Disasi',
  'Raphaël Varane',
  'Jules Koundé',
  'William Saliba',
  'Dayot Upamecano',
  'Lucas Hernandez',
  'Théo Hernandez',
  'Ibrahima Konaté'],
 'milieux': ['Mattéo Guendouzi',
  'Aurélien Tchouaméni',
  'Youssouf Fofana',
  'Adrien Rabiot',
  'Jordan Veretout',
  'Eduardo Camavinga'],
 'attaquants': ['Antoine Griezmann',
  'Olivier Giroud',
  'Kylian Mbappé',
  'Ousmane Dembélé',
  'Randal Kolo Muani',
  'Kingsley Coman',
  'Marcus Thuram',
  'Karim Benzema']}

In [268]:
équipes["France"]["gardiens"]

['Hugo Lloris', 'Steve Mandanda', 'Alphonse Areola']

In [253]:
liste_gardiens = [Joueur(j, joueurs[j]["nationalité"], joueurs[j]["rating"], joueurs[j]["âge"]) for j in g]
liste_défenseurs = [Joueur(j, joueurs[j]["nationalité"], joueurs[j]["rating"], joueurs[j]["âge"]) for j in d]
liste_milieux = [Joueur(j, joueurs[j]["nationalité"], joueurs[j]["rating"], joueurs[j]["âge"]) for j in m]
liste_attaquants = [Joueur(j, joueurs[j]["nationalité"], joueurs[j]["rating"], joueurs[j]["âge"]) for j in a]

In [254]:
france = Equipe("France", liste_gardiens, liste_défenseurs, liste_milieux, liste_attaquants)

In [308]:
france.rang

82.03846153846153

In [271]:
def créer_équipe(nom):
    
    l_g = [Joueur(j, joueurs[j]["nationalité"], joueurs[j]["rating"], joueurs[j]["âge"]) for j in équipes[nom]["gardiens"]]
    l_d = [Joueur(j, joueurs[j]["nationalité"], joueurs[j]["rating"], joueurs[j]["âge"]) for j in équipes[nom]["défenseurs"]]
    l_m = [Joueur(j, joueurs[j]["nationalité"], joueurs[j]["rating"], joueurs[j]["âge"]) for j in équipes[nom]["milieux"]]
    l_a = [Joueur(j, joueurs[j]["nationalité"], joueurs[j]["rating"], joueurs[j]["âge"]) for j in équipes[nom]["attaquants"]]
    
    return Equipe(nom, l_g, l_d, l_m, l_a)

In [274]:
france = créer_équipe("France")

In [275]:
allemagne = créer_équipe("Allemagne")

In [279]:
for j in france.selection():
    print(j.name)

Hugo Lloris
Théo Hernandez
Raphaël Varane
Jules Koundé
Lucas Hernandez
Aurélien Tchouaméni
Mattéo Guendouzi
Adrien Rabiot
Youssouf Fofana
Kylian Mbappé
Karim Benzema


In [280]:
for j in allemagne.selection():
    print(j.name)

Manuel Neuer
Antonio Rüdiger
Niklas Süle
Matthias Ginter
Nico Schlotterbeck
Joshua Kimmich
Leon Goretzka
Thomas Müller
Serge Gnabry
Kai Havertz
Mario Götze


In [485]:
class Match:
    def __init__(self, A, B):
        self.A = A
        self.B = B
        self.rating_A = sum([j.rating for j in self.A]) / len(self.A)
        self.rating_B = sum([j.rating for j in self.B]) / len(self.B)
        
    def play(self):
        coeff = 25
        diff = (self.rating_B - self.rating_A) / coeff
        return (1 - tanh(diff)) / 2

In [486]:
m = Match(france.selection(), allemagne.selection())

In [487]:
m.play()

0.4781956558768035

In [488]:
rating_A = 70.81818181818181
rating_B = 84.27272727272727
total = rating_A + rating_B
prob_A = rating_A / total
prob_B = 1 - prob_A
r = random.random()
print(r)
print(prob_A, prob_B)
if r > prob_A:
    print(f"B à gagné avec un proba de {prob_B}")
else:
    print(f"A à gagné avec un proba de {prob_A}")

0.3931966180565757
0.45662368112543966 0.5433763188745604
A à gagné avec un proba de 0.45662368112543966


In [489]:
l_équipes = []
for e in équipes:
    l_équipes.append(créer_équipe(e))

In [490]:
for e in l_équipes:
    print(e.nom, sum([j.rating for j in e.selection()]) / len(e.selection()))

Qatar 70.81818181818181
Equateur 73.81818181818181
Sénégal 78.81818181818181
Pays-Bas 83.36363636363636
Angleterre 84.81818181818181
Iran 73.54545454545455
Etats-Unis 76.45454545454545
Pays de Galles 73.63636363636364
Argentine 83.63636363636364
Arabie saoudite 71.45454545454545
Mexique 78.18181818181819
Pologne 78.9090909090909
France 84.27272727272727
Australie 72.72727272727273
Danemark 80.0
Tunisie 72.63636363636364
Espagne 83.9090909090909
Costa Rica 73.45454545454545
Allemagne 85.36363636363636
Japon 75.72727272727273
Belgique 82.9090909090909
Canada 74.0909090909091
Maroc 79.36363636363636
Croatie 81.63636363636364
Brésil 84.54545454545455
Serbie 79.72727272727273
Suisse 79.0909090909091
Cameroun 75.27272727272727
Portugal 84.9090909090909
Ghana 76.9090909090909
Uruguay 81.0909090909091
Corée du Sud 75.0909090909091


In [491]:
rating_A = 84
rating_B = 70
dif = (rating_B - rating_A) / 25
print((1 - tanh(dif) ) / 2)

0.753988716448948


In [497]:
eq1 = créer_équipe("Qatar")
for e in l_équipes:
    m = Match(eq1.selection(), e.selection())
    print(f"{eq1.nom} a {round(m.play()*100, 2)}% de chance de gagner contre {e.nom}")

Qatar a 50.0% de chance de gagner contre Qatar
Qatar a 44.03% de chance de gagner contre Equateur
Qatar a 34.52% de chance de gagner contre Sénégal
Qatar a 26.82% de chance de gagner contre Pays-Bas
Qatar a 24.6% de chance de gagner contre Angleterre
Qatar a 44.57% de chance de gagner contre Iran
Qatar a 38.91% de chance de gagner contre Etats-Unis
Qatar a 44.39% de chance de gagner contre Pays de Galles
Qatar a 26.4% de chance de gagner contre Argentine
Qatar a 48.73% de chance de gagner contre Arabie saoudite
Qatar a 35.68% de chance de gagner contre Mexique
Qatar a 34.36% de chance de gagner contre Pologne
Qatar a 25.42% de chance de gagner contre France
Qatar a 46.19% de chance de gagner contre Australie
Qatar a 32.42% de chance de gagner contre Danemark
Qatar a 46.37% de chance de gagner contre Tunisie
Qatar a 25.97% de chance de gagner contre Espagne
Qatar a 44.75% de chance de gagner contre Costa Rica
Qatar a 23.8% de chance de gagner contre Allemagne
Qatar a 40.31% de chance de

In [502]:
france.joueurs

#### Modèle de probabilité de gagner un match
(1-tanh(x-y))/2